El paquete de _Python_ [scikit-learn](http://scikit-learn.org) (_sklearn_ en lo que sigue) proporciona un marco de trabajo para el aprendizaje automático.

# Aprendizaje supervisado

Para ilustrar el concepto de aprendizaje supervisado vamos a usar el conjunto de datos [_Car Evaluation_](http://archive.ics.uci.edu/ml/datasets/Car+Evaluation) del repositorio [UCI](http://archive.ics.uci.edu/ml/). Este conjunto de datos contiene información acerca de la idoneidad de una serie de coches, en función de los siguientes atributos:
* _buying_: precio de compra. Posibles valores: `vhigh`, `high`, `med`, `low`.
* _maint_: coste de mantenimiento. Posibles valores: `vhigh`, `high`, `med`, `low`.
* _doors_: número de puertas. Posibles valores: `2`, `3`, `4`, `5more`.
* _persons_: número de asientos. Posibles valores: `2`, `4`, `more`.
* _lug\_boot_: tamaño del maletero. Posibles valores: `small`, `med`, `big`.
* _safety_: nivel de seguridad estimada. Posibles valores: `low`, `med`, `high`.

La idoneidad de cada coche se indica mediante el atributo _acceptability_, que los clasifica como `unacc`, `acc`, `good` o `vgood`.

Para leer los datos desde el fichero `cars.csv` que se proporciona se pueden evaluar las siguientes expresiones ([_Pandas_](http://pandas.pydata.org/) y [_NumPy_](http://www.numpy.org/) son paquetes de _Python_ para análisis de datos y cálculo científico, respectivamente):

In [ ]:
import pandas
import numpy

cars = pandas.read_csv('cars.csv', header=None,
                       names=['buying', 'maint', 'doors', 'persons',
                              'lug_boot', 'safety', 'acceptability'])
print(cars.shape)  # Número de filas y columnas
cars.head(10)  # 10 primeras filas

_sklearn_ no puede trabajar directamente con el conjunto de datos anterior, ya que asume que los valores de las variables discretas están codificadas con números enteros. Para transformar los datos a un formato adecuado ofrece diversas operaciones de preprocesamiento, entre las que se encuentran `OrdinalEncoder`, para codificar los atributos, y `LabelEncoder`, para codificar la variable objetivo.

In [ ]:
from sklearn import preprocessing

atributos = cars.loc[:, 'buying':'safety']  # selección de las columnas de atributos
objetivo = cars['acceptability']  # selección de la columna objetivo

# Para realizar una codificación de los datos, se crea una instancia del tipo de codificación pretendida y se ajusta a los datos disponibles mediante el método
# fit. Los métodos transform e inverse_transform permiten entonces codificar y descodificar, respectivamente, los datos.

In [ ]:
# El codificador adecuado para los atributos es OrdinalEncoder, ya que permite trabajar con el array completo de valores de los atributos.
codificador_atributos = preprocessing.OrdinalEncoder()
codificador_atributos.fit(atributos)
print(codificador_atributos.categories_)  # Categorías detectadas por el codificador para cada atributo
atributos_codificados = codificador_atributos.transform(atributos)
print(atributos_codificados)
print(codificador_atributos.inverse_transform([[3., 3., 0., 0., 2., 1.],
                                               [1., 1., 3., 2., 0., 1.]]))

In [ ]:
# El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja con la lista de sus valores, en lugar de con un array.
codificador_objetivo = preprocessing.LabelEncoder()
objetivo_codificado = codificador_objetivo.fit_transform(objetivo)  # El método fit_transform ajusta la codificación y la aplica a los datos a continuación
print(codificador_objetivo.classes_)  # Clases detectadas por el codificador para la variable objetivo
print(objetivo_codificado)
print(codificador_objetivo.inverse_transform([2, 1, 3]))

Una vez codificadas las variables, es necesario separar el conjunto de datos en dos: un conjunto de entrenamiento, que se usará para generar los distintos modelos; y un conjunto de prueba, que se usará para comparar los distintos modelos.

Un detalle a tener en cuenta es que la distribución de ejemplos en las distintas clases de aceptabilidad no es uniforme: hay 1210 coches (un 70.023 % del total) clasificados como inaceptables (`unacc`), 384 coches (22.222 %) clasificados como aceptables (`acc`), 69 coches (3.993 %) clasificados como buenos (`good`) y 65 coches (3.762 %) clasificados como muy buenos (`vgood`).

Es conveniente, por tanto, que la separación de los ejemplos se realice de manera estratificada, es decir, intentando mantener la proporción anterior tanto en el conjunto de entrenamiento como en el de prueba.

Para dividir un conjunto de datos en un subconjunto de entrenamiento y otro de prueba, _sklearn_ proporciona la función `train_test_split`.

In [ ]:
from sklearn import model_selection

print(cars.shape[0])  # Cantidad total de ejemplos
print(pandas.Series(objetivo).value_counts(normalize=True))  # Frecuencia total de cada clase de aceptabilidad

atributos_entrenamiento, atributos_prueba, objetivo_entrenamiento, objetivo_prueba = model_selection.train_test_split(
    atributos_codificados, objetivo_codificado,  # Conjuntos de datos a dividir, usando los mismos índices para ambos
    random_state=12345,  # Valor de la semilla aleatoria, para que el muestreo sea reproducible, a pesar de ser aleatorio
    test_size=.33,  # Tamaño del conjunto de prueba
    stratify=objetivo_codificado)  # Estratificamos respecto a la distribución de valores en la variable objetivo

# Comprobamos que el conjunto de prueba contiene el 33 % de los datos, en la misma proporción
# con respecto a la variable objetivo
print(atributos_prueba.shape[0],
      len(objetivo_prueba),
      1728 * .33)
print(pandas.Series(codificador_objetivo.inverse_transform(objetivo_prueba)).value_counts(normalize=True))

# Comprobamos que el conjunto de entrenamiento contiene el resto de los datos, en la misma
# proporción con respecto a la variable objetivo
print(atributos_entrenamiento.shape[0],
      len(objetivo_entrenamiento),
      1728 * .67)
print(pandas.Series(objetivo_entrenamiento).value_counts(normalize=True))

Para realizar aprendizaje supervisado en _sklearn_ basta crear una instancia de la clase de objetos que implemente el modelo que se quiera utilizar (árboles de decisión, _naive_ Bayes, _kNN_, etc.).

Cada una de estas instancias dispondrá de los siguientes métodos:
* El método `fit` permite entrenar el modelo, dados __por separado__ el conjunto de ejemplos de entrenamiento y la clase de cada uno de estos ejemplos.
* El método `predict` permite clasificar un nuevo ejemplo una vez entrenado el modelo.
* El método `score` calcula el rendimiento del modelo, dados __por separado__ el conjunto de ejemplos de prueba y la clase de cada uno de estos ejemplos.

### Árboles de decisión

_sklearn_ implementa los árboles de decisión clasificadores como instancias de la clase `DecisionTreeClassifier`.

Desafortunadamente, son árboles de decisión binarios construidos asumiendo atributos continuos y mediante un algoritmo distinto a _ID3_, que no está implementado.

En http://scikit-learn.org/stable/modules/tree.html se puede encontrar información acerca de los árboles de decisión implementados en _sklearn_.

### _Naive_ Bayes

_sklearn_ implementa _naive_ Bayes para atributos categóricos mediante instancias de la clase `CategoricalNB`.

In [ ]:
from sklearn import naive_bayes

clasif_NB = naive_bayes.CategoricalNB(alpha=1.0)  # alpha es el parámetro de suavizado
clasif_NB.fit(atributos_entrenamiento, objetivo_entrenamiento)

Las siguientes expresiones muestran las cuentas realizadas y (los logaritmos de) las probabilidades aprendidas por el modelo.

In [ ]:
print(clasif_NB.class_count_)
print(clasif_NB.class_log_prior_)
print(clasif_NB.category_count_)
print(clasif_NB.feature_log_prob_)

El método `predict` devuelve la clase predicha por el modelo para un nuevo ejemplo y el método `score` la exactitud (_accuracy_) media sobre un conjunto de datos de prueba.

In [ ]:
nuevos_ejemplos = [['vhigh', 'vhigh', '3', 'more', 'big', 'high'],
                   ['high', 'low', '3', '2', 'med', 'med']]
codificador_objetivo.inverse_transform(clasif_NB.predict(codificador_atributos.transform(nuevos_ejemplos)))

In [ ]:
# Calculamos la fracción de clases correctamente predichas para el conjunto de datos de prueba
clasif_NB.score(atributos_prueba, objetivo_prueba)

### kNN

_sklearn_ implementa _kNN_ como instancias de la clase `KNeighborsClassifier`. En http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html se puede encontrar una descripción de las distancias actualmente implementadas que se podrían usar.

In [ ]:
from sklearn import neighbors

clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=5, metric='hamming')

Entrenamos el modelo.

In [ ]:
clasif_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)

El método `kneighbors` permite encontrar los (índices de los) $k$ vecinos más cercanos de los ejemplos proporcionados, así como las distancias a las que se encuentran.

In [ ]:
distancias, vecinos = clasif_kNN.kneighbors(codificador_atributos.transform(nuevos_ejemplos))

# Vecinos más cercanos y distancia a ellos del primer ejemplo nuevo
print(nuevos_ejemplos[0])
print(codificador_atributos.inverse_transform(atributos_entrenamiento[vecinos[0]]))
print(distancias[0])
print(codificador_objetivo.inverse_transform(objetivo_entrenamiento[vecinos[0]]))

# Vecinos más cercanos y distancia a ellos del segundo ejemplo nuevo
print(nuevos_ejemplos[1])
print(codificador_atributos.inverse_transform(atributos_entrenamiento[vecinos[1]]))
print(distancias[1])
print(codificador_objetivo.inverse_transform(objetivo_entrenamiento[vecinos[1]]))

El método `predict` devuelve la clase predicha por el modelo para un nuevo ejemplo y el método `score` la exactitud (_accuracy_) media sobre un conjunto de datos de prueba.

In [ ]:
codificador_objetivo.inverse_transform(clasif_kNN.predict(codificador_atributos.transform(nuevos_ejemplos)))

In [ ]:
clasif_kNN.score(atributos_prueba, objetivo_prueba)

## Solicitudes de admisión en guarderías

El fichero de datos `nursery.csv` proporciona un conjunto de datos acerca de la evaluación de solicitudes de admisión en guarderías, en función de los siguientes atributos:
* _parents_, con posibles valores: `usual`, `pretentious`, `great_pret`.
* _has\_nurs_, con posibles valores: `proper`, `less_proper`, `improper`, `critical`, `very_crit`.
* _form_, con posibles valores: `complete`, `completed`, `incomplete`, `foster`.
* _children_, con posibles valores: `1`, `2`, `3`, `more`.
* _housing_, con posibles valores: `convenient`, `less_conv`, `critical`.
* _finance_, con posibles valores: `convenient`, `inconv`.
* _social_, con posibles valores: `non-prob`, `slightly_prob`, `problematic`.
* _health_, con posibles valores: `recommended`, `priority`, `not_recom`.

Los datos provienen de un sistema experto de decisión usado durante varios años de la década de los 80 en Liubliana (Eslovenia), que se desarrolló para poder proporcionar una explicación objetiva a las solicitudes rechazadas.

La evaluación de cada solicitud se indica mediante el atributo _evaluation_, que los clasifica como `not_recom`, `recommend`, `very_recom`, `priority` o `spec_prior`.

El objetivo es aprender a partir de los datos un modelo que prediga de la mejor forma posible cómo se evaluará una solicitud de admisión a partir de los valores de los atributos anteriores. Para ello se pide seguir los siguientes pasos:

* Leer los datos a partir del fichero `nursery.csv`.

* Codificar los datos con números enteros.

* Dividir el conjunto de datos en un subconjunto de entrenamiento (80 % de los datos) y un subconjunto de prueba (20 % de los datos). El primero de ellos se utilizará tanto para seleccionar un tipo de modelo utilizando la técnica de validación cruzada, como para entrenar el modelo finalmente seleccionado. El segundo se utilizará para medir la capacidad de generalización de este último.

* Usando el subconjunto de entrenamiento y utilizando la técnica de validación cruzada con 10 particiones, estimar la exactitud (_accuracy_) media de un modelo de tipo _naive_ Bayes con suavizado ɑ, para cada uno de los valores ɑ = 1, ..., 10, y de un modelo de tipo kNN, para cada uno de los valores k = 1, ..., 10.

__Nota__: la función `cross_val_score` del módulo `model_selection` de _sklearn_ implementa el procedimiento de validación cruzada. Admite, entre otros, los siguientes argumentos:
* _estimator_: modelo a evaluar.
* _X_: array con los valores de los atributos de los ejemplos.
* _y_: array con las clasificaciones de los ejemplos.
* _cv_: número de subconjuntos en los que dividir los datos.

Devuelve un array con el rendimiento del modelo sobre cada subconjunto, una vez entrenado con los ejemplos del resto de subconjuntos.

Para más información acerca de la implementación en _sklearn_ del método de validación cruzada puede consultarse http://scikit-learn.org/stable/modules/cross_validation.html.

* Con el mejor valor de suavizado determinado anteriormente, entrenar el algoritmo _naive_ Bayes con el subconjunto de entrenamiento y calcular la exactitud media sobre el subconjunto de prueba.

* Con el mejor número de vecinos determinado anteriormente, entrenar el algoritmo _kNN_ con el subconjunto de entrenamiento y calcular la exactitud media sobre el subconjunto de prueba.

* ¿Cuál de los dos modelos construidos en los puntos anteriores realiza mejores predicciones acerca de la evaluación de las solicitudes de admisión?